# Thanks to https://www.kaggle.com/siavrez/wavenet-keras and Sergey Bryansky.
# You can take a look at Sergey's kernel [here](https://www.kaggle.com/sggpls/shifted-rfc-pipeline) or [here](https://www.kaggle.com/sggpls/wavenet-with-shifted-rfc-proba). Also, Sergey's [data is here.](https://www.kaggle.com/sggpls/ion-shifted-rfc-proba)

In [11]:
import tensorflow as tf
import plaidml.keras
plaidml.keras.install_backend()
import os
os.environ['KERAS_BACKEND'] = 'plaidml.keras.backend'

import keras
from keras.layers import *
import pandas as pd
import numpy as np
import random
from keras.callbacks import Callback, LearningRateScheduler, EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras import backend as K
from keras import losses, models, optimizers
import gc

from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 500)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [9]:
# configurations and main hyperparammeters
EPOCHS = 110
NNBATCHSIZE = 16
GROUP_BATCH_SIZE = 4000
SEED = 321
LR = 0.001
SPLITS = 5

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [13]:
# read data
def read_data():
    train = pd.read_csv('./data/train_clean.csv',
                        dtype={
                            'time': np.float32,
                            'signal': np.float32,
                            'open_channels': np.int32
                        })
    test = pd.read_csv('./data/test_clean.csv',
                       dtype={
                           'time': np.float32,
                           'signal': np.float32
                       })
    sub = pd.read_csv('./data/sample_submission.csv',
                      dtype={'time': np.float32})

    Y_train_proba = np.load("./data/Y_train_proba.npy")
    Y_test_proba = np.load("./data/Y_test_proba.npy")

    for i in range(11):
        train[f"proba_{i}"] = Y_train_proba[:, i]
        test[f"proba_{i}"] = Y_test_proba[:, i]

    return train, test, sub


# create batches of 4000 observations
def batching(df, batch_size):
    df['group'] = df.groupby(df.index // batch_size,
                             sort=False)['signal'].agg(['ngroup']).values
    df['group'] = df['group'].astype(np.uint16)
    return df


# normalize the data (standard scaler). We can also try other scalers for a better score!
def normalize(train, test):
    train_input_mean = train.signal.mean()
    train_input_sigma = train.signal.std()
    train['signal'] = (train.signal - train_input_mean) / train_input_sigma
    test['signal'] = (test.signal - train_input_mean) / train_input_sigma
    return train, test


# get lead and lags features
def lag_with_pct_change(df, windows):
    for window in windows:
        df['signal_shift_pos_' +
           str(window)] = df.groupby('group')['signal'].shift(window).fillna(0)
        df['signal_shift_neg_' +
           str(window)] = df.groupby('group')['signal'].shift(-1 *
                                                              window).fillna(0)
    return df


# main module to run feature engineering. Here you may want to try and add other features and check if your score imporves :).
def run_feat_engineering(df, batch_size):
    # create batches
    df = batching(df, batch_size=batch_size)
    # create leads and lags (1, 2, 3 making them 6 features)
    df = lag_with_pct_change(df, [1, 2, 3])
    # create signal ** 2 (this is the new feature)
    df['signal_2'] = df['signal']**2
    return df


# fillna with the mean and select features for training
def feature_selection(train, test):
    features = [
        col for col in train.columns
        if col not in ['index', 'group', 'open_channels', 'time']
    ]
    train = train.replace([np.inf, -np.inf], np.nan)
    test = test.replace([np.inf, -np.inf], np.nan)
    for feature in features:
        feature_mean = pd.concat([train[feature], test[feature]],
                                 axis=0).mean()
        train[feature] = train[feature].fillna(feature_mean)
        test[feature] = test[feature].fillna(feature_mean)
    return train, test, features


# model function (very important, you can try different arquitectures to get a better score. I believe that top public leaderboard is a 1D Conv + RNN style)
def Classifier(shape_):
    def cbr(x, out_layer, kernel, stride, dilation):
        x = Conv1D(out_layer,
                   kernel_size=kernel,
                   dilation_rate=dilation,
                   strides=stride,
                   padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x

    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters=filters, kernel_size=1, padding='same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters=filters,
                              kernel_size=kernel_size,
                              padding='same',
                              activation='tanh',
                              dilation_rate=dilation_rate)(x)
            sigm_out = Conv1D(filters=filters,
                              kernel_size=kernel_size,
                              padding='same',
                              activation='sigmoid',
                              dilation_rate=dilation_rate)(x)
            x = Multiply()([tanh_out, sigm_out])
            x = Conv1D(filters=filters, kernel_size=1, padding='same')(x)
            res_x = Add()([res_x, x])
        return res_x

    inp = Input(shape=(shape_))
    x = cbr(inp, 64, 7, 1, 1)
    x = BatchNormalization()(x)
    x = wave_block(x, 16, 3, 12)
    x = BatchNormalization()(x)
    x = wave_block(x, 32, 3, 8)
    x = BatchNormalization()(x)
    x = wave_block(x, 64, 3, 4)
    x = BatchNormalization()(x)
    x = wave_block(x, 128, 3, 1)
    x = cbr(x, 32, 7, 1, 1)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    out = Dense(11, activation='softmax', name='out')(x)

    model = models.Model(inputs=inp, outputs=out)

    opt = Adam(lr=LR)
#     opt = tfa.optimizers.SWA(opt)
    model.compile(loss=categorical_crossentropy,
                  optimizer=opt,
                  metrics=['accuracy'])
    return model


# function that decrease the learning as epochs increase (i also change this part of the code)
def lr_schedule(epoch):
    if epoch < 30:
        lr = LR
    elif epoch < 40:
        lr = LR / 3
    elif epoch < 50:
        lr = LR / 5
    elif epoch < 60:
        lr = LR / 7
    elif epoch < 70:
        lr = LR / 9
    elif epoch < 80:
        lr = LR / 11
    elif epoch < 90:
        lr = LR / 13
    else:
        lr = LR / 100
    return lr


# class to get macro f1 score. This is not entirely necessary but it's fun to check f1 score of each epoch (be carefull, if you use this function early stopping callback will not work)
class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)

    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        score = f1_score(self.targets, pred, average='macro')
        print(f'F1 Macro Score: {score:.5f}')


# main function to perfrom groupkfold cross validation (we have 1000 vectores of 4000 rows and 8 features (columns)). Going to make 5 groups with this subgroups.
def run_cv_model_by_batch(train, test, splits, batch_col, feats,
                          sample_submission, nn_epochs, nn_batch_size):

    seed_everything(SEED)
    K.clear_session()
    config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                                      inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),
                                config=config)
    tf.compat.v1.keras.backend.set_session(sess)
    oof_ = np.zeros(
        (len(train), 11)
    )  # build out of folds matrix with 11 columns, they represent our target variables classes (from 0 to 10)
    preds_ = np.zeros((len(test), 11))
    target = ['open_channels']
    group = train['group']
    kf = GroupKFold(n_splits=5)
    splits = [x for x in kf.split(train, train[target], group)]

    new_splits = []
    for sp in splits:
        new_split = []
        new_split.append(np.unique(group[sp[0]]))
        new_split.append(np.unique(group[sp[1]]))
        new_split.append(sp[1])
        new_splits.append(new_split)
    # pivot target columns to transform the net to a multiclass classification estructure (you can also leave it in 1 vector with sparsecategoricalcrossentropy loss function)
    tr = pd.concat([pd.get_dummies(train.open_channels), train[['group']]],
                   axis=1)

    tr.columns = ['target_' + str(i) for i in range(11)] + ['group']
    target_cols = ['target_' + str(i) for i in range(11)]
    train_tr = np.array(
        list(tr.groupby('group').apply(
            lambda x: x[target_cols].values))).astype(np.float32)
    train = np.array(
        list(train.groupby('group').apply(lambda x: x[feats].values)))
    test = np.array(
        list(test.groupby('group').apply(lambda x: x[feats].values)))

    for n_fold, (tr_idx, val_idx, val_orig_idx) in enumerate(new_splits[0:],
                                                             start=0):
        train_x, train_y = train[tr_idx], train_tr[tr_idx]
        valid_x, valid_y = train[val_idx], train_tr[val_idx]
        print(f'Our training dataset shape is {train_x.shape}')
        print(f'Our validation dataset shape is {valid_x.shape}')

        gc.collect()
        shape_ = (
            None, train_x.shape[2]
        )  # input is going to be the number of feature we are using (dimension 2 of 0, 1, 2)
        model = Classifier(shape_)
        # using our lr_schedule function
        cb_lr_schedule = LearningRateScheduler(lr_schedule)
        model.fit(
            train_x,
            train_y,
            epochs=nn_epochs,
            callbacks=[
                cb_lr_schedule,
                EarlyStopping(monitor='val_accuracy', mode='max', verbose=1)
            ],  # adding custom evaluation metric for each epoch
            batch_size=nn_batch_size,
            verbose=2,
            validation_data=(valid_x, valid_y))
        
        preds_f = model.predict(valid_x)
        f1_score_ = f1_score(
            np.argmax(valid_y, axis=2).reshape(-1),
            np.argmax(preds_f, axis=2).reshape(-1),
            average='macro'
        )  # need to get the class with the biggest probability
        print(
            f'Training fold {n_fold + 1} completed. macro f1 score : {f1_score_ :1.5f}'
        )
        preds_f = preds_f.reshape(-1, preds_f.shape[-1])
        oof_[val_orig_idx, :] += preds_f
        te_preds = model.predict(test)
        te_preds = te_preds.reshape(-1, te_preds.shape[-1])
        preds_ += te_preds / SPLITS
    # calculate the oof macro f1_score
    f1_score_ = f1_score(
        np.argmax(train_tr, axis=2).reshape(-1),
        np.argmax(oof_, axis=1),
        average='macro'
    )  # axis 2 for the 3 Dimension array and axis 1 for the 2 Domension Array (extracting the best class)
    print(f'Training completed. oof macro f1 score : {f1_score_:1.5f}')
    sample_submission['open_channels'] = np.argmax(preds_, axis=1).astype(int)
    sample_submission.to_csv('submission_wavenet.csv',
                             index=False,
                             float_format='%.4f')


# this function run our entire program
def run_everything():

    print('Reading Data Started...')
    train, test, sample_submission = read_data()
    train, test = normalize(train, test)
    print('Reading and Normalizing Data Completed')

    print('Creating Features')
    print('Feature Engineering Started...')
    train = run_feat_engineering(train, batch_size=GROUP_BATCH_SIZE)
    test = run_feat_engineering(test, batch_size=GROUP_BATCH_SIZE)
    train, test, features = feature_selection(train, test)
    print('Feature Engineering Completed...')

    print(
        f'Training Wavenet model with {SPLITS} folds of GroupKFold Started...')
    run_cv_model_by_batch(train, test, SPLITS, 'group', features,
                          sample_submission, EPOCHS, NNBATCHSIZE)
    print('Training completed...')


run_everything()

Reading Data Started...
Reading and Normalizing Data Completed
Creating Features
Feature Engineering Started...
Feature Engineering Completed...
Training Wavenet model with 5 folds of GroupKFold Started...
Our training dataset shape is (1000, 4000, 19)
Our validation dataset shape is (250, 4000, 19)


INFO:plaidml:Opening device "metal_amd_radeon_pro_560x.0"


Train on 1000 samples, validate on 250 samples
Epoch 1/110


INFO:plaidml:Analyzing Ops: 2102 of 3912 operations complete
INFO:plaidml:Analyzing Ops: 3792 of 3912 operations complete


 - 109s - loss: 0.5880 - acc: 0.8319 - val_loss: 0.8162 - val_acc: 0.8614
Epoch 2/110
 - 71s - loss: 0.1999 - acc: 0.9529 - val_loss: 0.4384 - val_acc: 0.9453
Epoch 3/110
 - 72s - loss: 0.1563 - acc: 0.9623 - val_loss: 0.2110 - val_acc: 0.9600
Epoch 4/110
 - 72s - loss: 0.1353 - acc: 0.9648 - val_loss: 0.1476 - val_acc: 0.9631
Epoch 5/110
 - 72s - loss: 0.1261 - acc: 0.9658 - val_loss: 0.1216 - val_acc: 0.9654
Epoch 6/110
 - 72s - loss: 0.1230 - acc: 0.9659 - val_loss: 0.1162 - val_acc: 0.9648
Epoch 7/110
 - 71s - loss: 0.1216 - acc: 0.9658 - val_loss: 0.1121 - val_acc: 0.9656
Epoch 8/110
 - 71s - loss: 0.1235 - acc: 0.9655 - val_loss: 0.1069 - val_acc: 0.9659
Epoch 9/110
 - 71s - loss: 0.1127 - acc: 0.9667 - val_loss: 0.1036 - val_acc: 0.9661
Epoch 10/110
 - 71s - loss: 0.1118 - acc: 0.9666 - val_loss: 0.1059 - val_acc: 0.9651
Epoch 11/110
 - 71s - loss: 0.1124 - acc: 0.9665 - val_loss: 0.1013 - val_acc: 0.9662
Epoch 12/110
 - 73s - loss: 0.1078 - acc: 0.9669 - val_loss: 0.1004 - val_

...

 - 109s - loss: 0.6337 - acc: 0.8284 - val_loss: 1.0166 - val_acc: 0.8824
Epoch 2/110
 - 73s - loss: 0.2148 - acc: 0.9552 - val_loss: 0.6039 - val_acc: 0.9504
Epoch 3/110
 - 72s - loss: 0.1622 - acc: 0.9632 - val_loss: 0.3013 - val_acc: 0.9632
Epoch 4/110
 - 72s - loss: 0.1452 - acc: 0.9647 - val_loss: 0.1749 - val_acc: 0.9654
Epoch 5/110
 - 72s - loss: 0.1449 - acc: 0.9644 - val_loss: 0.1276 - val_acc: 0.9664
Epoch 6/110
 - 72s - loss: 0.1282 - acc: 0.9658 - val_loss: 0.1162 - val_acc: 0.9669
Epoch 7/110
 - 73s - loss: 0.1258 - acc: 0.9658 - val_loss: 0.1111 - val_acc: 0.9670
Epoch 8/110
 - 72s - loss: 0.1213 - acc: 0.9661 - val_loss: 0.1070 - val_acc: 0.9670
Epoch 9/110
 - 72s - loss: 0.1169 - acc: 0.9663 - val_loss: 0.1047 - val_acc: 0.9671
Epoch 10/110
 - 73s - loss: 0.1159 - acc: 0.9662 - val_loss: 0.1038 - val_acc: 0.9664
Epoch 11/110
 - 72s - loss: 0.1133 - acc: 0.9664 - val_loss: 0.0999 - val_acc: 0.9673
Epoch 12/110
 - 72s - loss: 0.1100 - acc: 0.9666 - val_loss: 0.0982 - val_

Epoch 97/110
 - 70s - loss: 0.0809 - acc: 0.9701 - val_loss: 0.0812 - val_acc: 0.9693
Epoch 98/110
 - 70s - loss: 0.0808 - acc: 0.9701 - val_loss: 0.0811 - val_acc: 0.9694
Epoch 99/110
 - 70s - loss: 0.0814 - acc: 0.9701 - val_loss: 0.0813 - val_acc: 0.9693
Epoch 100/110
 - 70s - loss: 0.0814 - acc: 0.9700 - val_loss: 0.0812 - val_acc: 0.9694
Epoch 101/110
 - 70s - loss: 0.0812 - acc: 0.9701 - val_loss: 0.0812 - val_acc: 0.9694
Epoch 102/110
 - 70s - loss: 0.0809 - acc: 0.9702 - val_loss: 0.0811 - val_acc: 0.9694
Epoch 103/110
 - 70s - loss: 0.0814 - acc: 0.9701 - val_loss: 0.0812 - val_acc: 0.9694
Epoch 104/110
 - 70s - loss: 0.0807 - acc: 0.9702 - val_loss: 0.0812 - val_acc: 0.9694
Epoch 105/110
 - 70s - loss: 0.0809 - acc: 0.9702 - val_loss: 0.0812 - val_acc: 0.9694
Epoch 106/110
 - 70s - loss: 0.0815 - acc: 0.9700 - val_loss: 0.0813 - val_acc: 0.9694
Epoch 107/110
 - 70s - loss: 0.0822 - acc: 0.9699 - val_loss: 0.0813 - val_acc: 0.9694
Epoch 108/110
 - 70s - loss: 0.0813 - acc: 0.9

INFO:plaidml:Analyzing Ops: 2557 of 3912 operations complete


 - 106s - loss: 0.5933 - acc: 0.8352 - val_loss: 0.9926 - val_acc: 0.8604
Epoch 2/110
 - 70s - loss: 0.2011 - acc: 0.9560 - val_loss: 0.5674 - val_acc: 0.9381
Epoch 3/110
 - 70s - loss: 0.1561 - acc: 0.9630 - val_loss: 0.2655 - val_acc: 0.9573
Epoch 4/110
 - 70s - loss: 0.1393 - acc: 0.9649 - val_loss: 0.1556 - val_acc: 0.9660
Epoch 5/110
 - 70s - loss: 0.1303 - acc: 0.9656 - val_loss: 0.1293 - val_acc: 0.9661
Epoch 6/110
 - 70s - loss: 0.1255 - acc: 0.9659 - val_loss: 0.1120 - val_acc: 0.9670
Epoch 7/110
 - 70s - loss: 0.1220 - acc: 0.9661 - val_loss: 0.1043 - val_acc: 0.9673
Epoch 8/110
 - 70s - loss: 0.1162 - acc: 0.9664 - val_loss: 0.1029 - val_acc: 0.9672
Epoch 9/110
 - 70s - loss: 0.1142 - acc: 0.9666 - val_loss: 0.1023 - val_acc: 0.9672
Epoch 10/110
 - 70s - loss: 0.1126 - acc: 0.9665 - val_loss: 0.1034 - val_acc: 0.9669
Epoch 11/110
 - 70s - loss: 0.1102 - acc: 0.9666 - val_loss: 0.0972 - val_acc: 0.9676
Epoch 12/110
 - 70s - loss: 0.1755 - acc: 0.9516 - val_loss: 0.1292 - val_

Epoch 97/110
 - 70s - loss: 0.0877 - acc: 0.9681 - val_loss: 0.0840 - val_acc: 0.9681
Epoch 98/110
 - 70s - loss: 0.0874 - acc: 0.9681 - val_loss: 0.0841 - val_acc: 0.9681
Epoch 99/110
 - 70s - loss: 0.0874 - acc: 0.9681 - val_loss: 0.0840 - val_acc: 0.9681
Epoch 100/110
 - 71s - loss: 0.0871 - acc: 0.9682 - val_loss: 0.0840 - val_acc: 0.9681
Epoch 101/110
 - 70s - loss: 0.0876 - acc: 0.9681 - val_loss: 0.0840 - val_acc: 0.9681
Epoch 102/110
 - 70s - loss: 0.0875 - acc: 0.9681 - val_loss: 0.0840 - val_acc: 0.9682
Epoch 103/110
 - 70s - loss: 0.0875 - acc: 0.9680 - val_loss: 0.0840 - val_acc: 0.9682
Epoch 104/110
 - 70s - loss: 0.0882 - acc: 0.9680 - val_loss: 0.0840 - val_acc: 0.9682
Epoch 105/110
 - 70s - loss: 0.0873 - acc: 0.9681 - val_loss: 0.0840 - val_acc: 0.9682
Epoch 106/110
 - 70s - loss: 0.0875 - acc: 0.9681 - val_loss: 0.0841 - val_acc: 0.9682
Epoch 107/110
 - 70s - loss: 0.0870 - acc: 0.9682 - val_loss: 0.0840 - val_acc: 0.9682
Epoch 108/110
 - 70s - loss: 0.0877 - acc: 0.9